In [1]:
import dask.dataframe as dd
import  pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import time
import gc 


In [2]:
import datatable as dt
start_time = time.time()
X = dt.fread('/home/carlchao/Riiid_data/data/train.csv')
X = X.to_pandas()
print("datatable took %s seconds" % (time.time() - start_time))


datatable took 7.857659101486206 seconds


In [3]:
start_time = time.time()
test_df = dt.fread('/home/carlchao/Riiid_data/data/example_test.csv')
test_df = test_df.to_pandas()
print("datatable took %s seconds" % (time.time() - start_time))



datatable took 0.003080606460571289 seconds


In [4]:
questions_df = pd.read_csv('/home/carlchao/Riiid_data/data/questions.csv',
                            usecols=[0,1, 3,4],
                            dtype={'question_id': 'int16',
                              'part': 'int8','bundle_id': 'int8','tags': 'str'}
                          )
tag = questions_df["tags"].str.split(" ", n = 10, expand = True) 
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']

questions_df =  pd.concat([questions_df,tag],axis=1)
questions_df['tags1'] = pd.to_numeric(questions_df['tags1'], errors='coerce')
questions_df['tags2'] = pd.to_numeric(questions_df['tags2'], errors='coerce')
questions_df['tags3'] = pd.to_numeric(questions_df['tags3'], errors='coerce')
questions_df['tags4'] = pd.to_numeric(questions_df['tags4'], errors='coerce')
questions_df['tags5'] = pd.to_numeric(questions_df['tags5'], errors='coerce')
questions_df['tags6'] = pd.to_numeric(questions_df['tags6'], errors='coerce')

In [5]:
X = X.sort_values(['timestamp'], ascending=True)
X.drop(['timestamp','content_type_id'], axis=1,   inplace=True)

In [6]:
X = X.iloc[:8000000].copy()
X['prior_question_had_explanation'].fillna(False, inplace=True)
# X = pd.merge(X, results_u, on=['user_id'], how="left")
# X = pd.merge(X, results_c, on=['content_id'], how="left")
X = pd.merge(X, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')

X=X[X.answered_correctly!= -1 ]
X=X.sort_values(['user_id'])
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)

In [7]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])
X = X[['bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc','tags1','tags2','tags3']] 
X.fillna(0.5,  inplace=True)

In [31]:
test_df = test_df.sort_values(['user_id','timestamp'], ascending=False)
test_df['answer_time'] = test_df.groupby(['user_id'])['prior_question_elapsed_time'].shift(1) 
test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')    
test_df['prior_question_had_explanation'].fillna(False, inplace=True)
test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
test_df = test_df[['bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc','tags1','tags2','tags3']]


In [28]:
test_df.iloc[0]

row_id                                         92
group_num                                       3
timestamp                             23890386630
user_id                                2103436554
content_id                                   1722
content_type_id                             False
task_container_id                            1033
prior_question_elapsed_time                 23333
prior_question_had_explanation               True
prior_group_answers_correct                      
prior_group_responses                            
answer_time                                   NaN
question_id                                  1722
bundle_id                                     -71
part                                            3
tags                                    113 38 29
tags1                                         113
tags2                                          38
tags3                                          29
tags4                                         NaN
